# Code

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from keras.layers import GRU

In [2]:
filename = '../Data/BERT2560/Combined/reshape_bb_2560.csv'

In [3]:
data = pd.read_csv(filename)

In [4]:
filename = filename[:filename.index('.')]

In [5]:
# data = data[:2000]
label = data['label']
features = data.drop(['uid', 'label'], axis=1)

In [6]:
del data

In [8]:
#features = features.to_numpy()
features = features.reshape(39259, 5, 768)

In [9]:
X, a, y, b = train_test_split(features, label, random_state=42, stratify=label, test_size=0.3)
print(X.shape, y.shape, a.shape, b.shape)

Xv, Xt, yv, yt = train_test_split(a, b, random_state=42, stratify=b, test_size=0.4)
print(Xv.shape, Xt.shape, yv.shape, yt.shape)

# Xv = X[-6000:]
# X = X[:-6000]

(27481, 5, 768) (27481,) (11778, 5, 768) (11778,)
(7066, 5, 768) (4712, 5, 768) (7066,) (4712,)


In [10]:
# defining the LSTM model
model = Sequential()
model.add(GRU(100, input_shape=(features.shape[1], features.shape[2]), return_sequences=True, activation='relu')) #bidirectional lstm
model.add(Dropout(0.25)) #do for both bid and normal lstm 
model.add(GRU(50,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

In [11]:
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [12]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model_gru_legal_8' + filename + '_.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [ ]:
history = model.fit(X, y, validation_data=(Xv, yv), epochs=1000, batch_size=8, callbacks=[es, mc])

Epoch 1/1000
3432/3436 [============================>.] - ETA: 0s - loss: 0.6820 - accuracy: 0.5586
Epoch 1: val_accuracy improved from -inf to 0.56850, saving model to best_model_gru_legal_8_.h5
3436/3436 [==============================] - 52s 12ms/step - loss: 0.6820 - accuracy: 0.5586 - val_loss: 0.6737 - val_accuracy: 0.5685
Epoch 2/1000
3433/3436 [============================>.] - ETA: 0s - loss: 0.6731 - accuracy: 0.5740
Epoch 2: val_accuracy improved from 0.56850 to 0.56963, saving model to best_model_gru_legal_8_.h5
3436/3436 [==============================] - 39s 11ms/step - loss: 0.6732 - accuracy: 0.5739 - val_loss: 0.6701 - val_accuracy: 0.5696
Epoch 3/1000
3435/3436 [============================>.] - ETA: 0s - loss: 0.6659 - accuracy: 0.5848
Epoch 3: val_accuracy improved from 0.56963 to 0.58336, saving model to best_model_gru_legal_8_.h5
3436/3436 [==============================] - 45s 13ms/step - loss: 0.6659 - accuracy: 0.5848 - val_loss: 0.6741 - val_accuracy: 0.5834
E

Epoch 28/1000
3429/3436 [============================>.] - ETA: 0s - loss: 0.5576 - accuracy: 0.7103
Epoch 28: val_accuracy did not improve from 0.65284
3436/3436 [==============================] - 28s 8ms/step - loss: 0.5577 - accuracy: 0.7103 - val_loss: 0.6513 - val_accuracy: 0.6339
Epoch 29/1000
3436/3436 [==============================] - ETA: 0s - loss: 0.5539 - accuracy: 0.7153
Epoch 29: val_accuracy did not improve from 0.65284
3436/3436 [==============================] - 29s 8ms/step - loss: 0.5539 - accuracy: 0.7153 - val_loss: 0.6368 - val_accuracy: 0.6477
Epoch 30/1000
3436/3436 [==============================] - ETA: 0s - loss: 0.5486 - accuracy: 0.7153
Epoch 30: val_accuracy improved from 0.65284 to 0.65440, saving model to best_model_gru_legal_8_.h5
3436/3436 [==============================] - 29s 8ms/step - loss: 0.5486 - accuracy: 0.7153 - val_loss: 0.6424 - val_accuracy: 0.6544
Epoch 31/1000
3433/3436 [============================>.] - ETA: 0s - loss: 0.5460 - accurac

In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
saved_model = load_model('best_model_gru_legal_8' + filename + '_.h5')
# evaluate the model
_, train_acc = saved_model.evaluate(X, y, verbose=0)
_, test_acc = saved_model.evaluate(Xt, yt, verbose=0)

print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

In [ ]:
lst = [train_acc, test_acc]
df = pd.DataFrame(lst, index =['train_acc', 'test_acc'])
df.to_csv(filename +'best_model_gru_legal_8.csv')

In [ ]:
# plot history
pyplot.plot(history.history['accuracy'], label='train')
pyplot.plot(history.history['val_accuracy'], label='test')
pyplot.legend()
pyplot.savefig(filename + 'best_model_gru_legal_8.png', bbox_inches='tight')
pyplot.show()


In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
pyplot.savefig(filename + 'best_model_gru_legal_8.png', bbox_inches='tight')
plt.show();

In [ ]:
from sklearn.metrics import confusion_matrix
yp = (saved_model.predict(X) > 0.5).astype('int32') 
matrix = confusion_matrix(y,yp, labels=[1,0])
cm = pd.DataFrame(matrix, index=['1','0'],columns=['1','0'])
from sklearn.metrics import accuracy_score
print("Test Accuracy Score: %.3f" %(accuracy_score(y,yp)))

from sklearn.metrics import recall_score
print("Test Recall Score: %.3f" %(recall_score(y,yp)))

from sklearn.metrics import precision_score
print("Test Precision Score: %.3f" %(precision_score(y,yp)))

from sklearn.metrics import f1_score
print("Test F1 Score: %.3f" %(f1_score(y,yp)))

In [ ]:
yp = (saved_model.predict(Xt) > 0.5).astype('int32') 
matrix = confusion_matrix(yt,yp, labels=[1,0])
cm = pd.DataFrame(matrix, index=['1','0'],columns=['1','0'])
from sklearn.metrics import accuracy_score
print("Test Accuracy Score: %.3f" %(accuracy_score(yt,yp)))

from sklearn.metrics import recall_score
print("Test Recall Score: %.3f" %(recall_score(yt,yp)))

from sklearn.metrics import precision_score
print("Test Precision Score: %.3f" %(precision_score(yt,yp)))

from sklearn.metrics import f1_score
print("Test F1 Score: %.3f" %(f1_score(yt,yp)))